In [ ]:
sessionInfo()
set.seed(1)
.libPaths()

## Imports

In [ ]:
library(devtools)
library(vctrs)
library(Seurat)
library(dplyr)
library(Matrix)
library(tidyr)
library(limma)
library(ggplot2)
library(ggthemes)
library(patchwork)
library(gprofiler2)
library(ggrepel)
library(scales)
library(ggthemes)
library(purrr)
library(MAST)
library(qpcR)
library(enrichR)
library(harmony)
library(Nebulosa)
library(celldex)
library(ggrastr)
library(ggupset)

## Custom Utility Functions

The figsize in R is specified in inches, 1 inch = 2.54 cm.

In [ ]:
set_figsize <- function(width, height){
    options(repr.plot.width = width, 
            repr.plot.height = height)
}

# Load in diffuse b cell lymphoma dataset

In [ ]:
counts <- read.table(file = 'pathway/Lymphom/diffuseb/GSE182434_raw_count_matrix.txt.gz')
head(counts)

In [ ]:
colnames(counts) <- counts[1, ]
rownames(counts) <- counts[, 1]
counts$Gene <- NULL
counts <- counts[2:nrow(counts), ]

In [ ]:
head(counts)

In [ ]:
diffuse_b <- CreateSeuratObject(counts = counts, project = "diffuseb", min.cells = 3, min.features = 200)

In [ ]:
object <- diffuse_b
object

In [ ]:
# add metadata
md <- read.table(file = 'pathway/Lymphom/diffuseb/GSE182434_cell_annotation.txt.gz')
colnames(md) <- md[1, ]
rownames(md) <- md[, 1]
md$ID <- NULL
md <- md[2:nrow(md), ]

object <- AddMetaData(object = object, metadata = md)
object
head(object@meta.data)

#unique values per column
sapply(object@meta.data, function(x) unique(x))

In [ ]:
obj <- object

In [ ]:
dir_plots <- './diffuseb/outs/'
dir.create(dir_plots)

# Process the seurat object, construct the umap

## QC

In [ ]:
# The number of features and UMIs (nFeature_RNA and nCount_RNA) are automatically calculated for every object by Seurat.
# For non-UMI data, nCount_RNA represents the sum of the non-normalized values within a cell
# We calculate the percentage of mitochondrial features here and store it in object metadata as `percent.mito`.
# We use raw count data since this represents non-transformed and non-log-normalized counts
# The % of UMI mapping to MT-features is a common scRNA-seq QC metric.
mito.features <- grep(pattern = "^MT-", x = rownames(x = obj), value = TRUE)
percent.mito <- Matrix::colSums(x = GetAssayData(object = obj, slot = 'counts')[mito.features, ]) / Matrix::colSums(x = GetAssayData(object = obj, slot = 'counts'))

In [ ]:
# The [[ operator can add columns to object metadata, and is a great place to stash QC stats
obj[['percent.mito']] <- percent.mito
plot <- VlnPlot(object = obj, features = c("nFeature_RNA"), ncol = 3, pt.size = 0.000001)
l1 <- plot$layers[[1]]
l2 <- plot$layers[[2]]

plot$layers[[1]] <- l2
plot$layers[[2]] <- l1

plot

plot <- VlnPlot(object = obj, features = c("nCount_RNA"), ncol = 3, pt.size = 0.000001)
l1 <- plot$layers[[1]]
l2 <- plot$layers[[2]]

plot$layers[[1]] <- l2
plot$layers[[2]] <- l1

plot

plot <- VlnPlot(object = obj, features = c("percent.mito"), ncol = 3, pt.size = 0.000001)
l1 <- plot$layers[[1]]
l2 <- plot$layers[[2]]

plot$layers[[1]] <- l2
plot$layers[[2]] <- l1

plot

In [ ]:
obj <- subset(obj, subset = percent.mito < 0.15)

In [ ]:
obj

# Integration

In [ ]:
obj@meta.data$sample <- obj@meta.data$Sample
unique(obj$sample)

In [ ]:
Sys.time()
obj.list <- SplitObject(obj, split.by = "sample")

In [ ]:
markers.remove <- grep(pattern = c("^TRAV|^TRBV|^TRGV|^TRDV|^IGKV|^IGLV|^IGHV|^IGHG|^IGK"),  x = rownames(x = obj), value = TRUE)

In [ ]:
obj.list <- lapply(X = obj.list, FUN = function(x) {
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method = "vst", nfeatures = 3000)
})

for (i in 1:length(obj.list)){
     VariableFeatures(obj.list[[i]]) <- VariableFeatures(object = obj.list[[i]])[!(VariableFeatures(object = obj.list[[i]])%in%markers.remove)]
}
Sys.time()

In [ ]:
features <- SelectIntegrationFeatures(object.list = obj.list, nfeatures = 2500)

In [ ]:
Sys.time()

In [ ]:
obj.list <- lapply(X = obj.list, FUN = function(x) {
    x <- ScaleData(x, features = features, verbose = FALSE)
    x <- RunPCA(x, features = features, verbose = FALSE)
})
Sys.time()

In [ ]:
obj.anchors <- FindIntegrationAnchors(object.list = obj.list, anchor.features = features, reduction = "rpca")
Sys.time()

In [ ]:
obj.integrated <- IntegrateData(anchorset = obj.anchors)
Sys.time()

In [ ]:
obj.integrated

In [ ]:
saveRDS(obj.integrated, file = './diffuseb/object_without_umap_integrated.rds')

# Working with integrated file

In [ ]:
obj.integrated <- ScaleData(obj.integrated, features = VariableFeatures(object = obj.integrated), vars.to.regress = c("nCount_RNA", "percent.mito"))
obj.integrated <- RunPCA(obj.integrated, features = VariableFeatures(object = obj.integrated))

In [ ]:
default_width <- 11
set_figsize(default_width, default_width)

In [ ]:
Idents(obj.integrated) <- 'sample'
DimPlot(obj.integrated, reduction = 'pca', label = TRUE)

In [ ]:
Idents(obj.integrated) <- 'sample'
VlnPlot(obj.integrated, "ACTB", pt.size = 0)

In [ ]:
# ProjectDim scores each feature in the dataset (including features not included in the PCA) based on their correlation 
# with the calculated components. Though we don't use this further here, it can be used to identify markers that 
# are strongly correlated with cellular heterogeneity, but may not have passed through variable feature selection. 
# The results of the projected PCA can be explored by setting `projected = TRUE`in the functions above
obj.integrated <- ProjectDim(object = obj.integrated)

In [ ]:
ElbowPlot(object = obj.integrated, ndims = 50)

In [ ]:
DimHeatmap(object = obj.integrated, dims = 1, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 2, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 3, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 4, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 5, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 6, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 7, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 8, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 9, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 10, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 11, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 12, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 13, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 14, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 15, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 16, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 17, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 18, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 19, cells = 500, balanced = TRUE)
DimHeatmap(object = obj.integrated, dims = 20, cells = 500, balanced = TRUE)

# Cluster the cells

In [ ]:
dim_number <- 30

In [ ]:
obj.integrated <- FindNeighbors(object = obj.integrated, dims = 1:dim_number)

In [ ]:
res <- 0.8

In [ ]:
obj.integrated <- FindClusters(object = obj.integrated, resolution = res)

# Run Non-linear dimensional reduction (UMAP)

In [ ]:
obj.integrated <- RunUMAP(obj.integrated, dims = 1:dim_number)

In [ ]:
obj <- obj.integrated

# Summ up metadata

In [ ]:
colnames(obj@meta.data)

In [ ]:
#look how many samples of tumor and normal in each cluster
obj@meta.data$summ_column <- paste0(obj@meta.data$CellType, '_', obj@meta.data$Tissue)
data.frame(table(obj@meta.data$summ_column))

In [ ]:
#look whether TumorNormal is related to all samples
obj@meta.data$summ_column <- paste0(obj@meta.data$Patient, '_', obj@meta.data$TumorNormal)
data.frame(table(obj@meta.data$summ_column))

In [ ]:
#look where the normal and tumor cells are distributed
obj@meta.data$summ_column <- paste0(obj@meta.data$CellType, '_', obj@meta.data$Tissue, '_', obj@meta.data$TumorNormal)
data.frame(table(obj@meta.data$summ_column))

In [ ]:
obj1 <- obj

In [ ]:
obj1

In [ ]:
#remove the "normal" cells from DLBCL and FL subset:
obj@meta.data$summ_column <- paste0(obj@meta.data$Patient, '_', obj@meta.data$TumorNormal)
filtering <- unique(obj@meta.data$summ_column)
filtering <- filtering[!(filtering %in% c('DLBCL002_Normal', 'FL2_Normal'))]
filtering
obj <- subset(obj, summ_column %in% filtering)
obj

# Plot UMAPs

In [ ]:
default_width <- 10

In [ ]:
colnames(obj@meta.data)

In [ ]:
unique(obj$CellType)

In [ ]:
obj$Tissue <- factor(obj$Tissue, levels = c('Tonsil', 'FL', 'DLBCL'))
obj$CellType <- factor(obj$CellType, levels = c('B cells', 'Plasma cells', 'Monocytes and Macrophages', 'T cells CD4', 'TFH', 'Tregs', 'T cells CD8', 'NK cells', 'Others'))

obj@meta.data$Tissue <- factor(obj@meta.data$Tissue, levels = c('Tonsil', 'FL', 'DLBCL'))
obj@meta.data$CellType <- factor(obj@meta.data$CellType, levels = c('B cells', 'Plasma cells', 'Monocytes and Macrophages', 'T cells CD4', 'TFH', 'Tregs', 'T cells CD8', 'NK cells', 'Others'))

In [ ]:
#plot umaps
width <- 10
height <- 10
set_figsize(width, height)
name <- 'umap_all'
grouping <- 'CellType'
umap_plot <- DimPlot(obj, reduction = 'umap', group.by = grouping, pt.size = 7, label = TRUE, repel  = TRUE, label.size = 7, raster = T, raster.dpi = c(5000, 5000)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          plot.title = element_text(size = 20, face = 'plain'),
          legend.text=element_text(size=20)) + NoAxes() + NoLegend()
print(umap_plot)
ggsave(umap_plot, file = paste0(dir_plots, name, '.pdf'), width = width, height = height)

umap_plot <- DimPlot(obj, reduction = 'umap', group.by = grouping, pt.size = 7, label = TRUE, repel  = TRUE, label.size = 7, raster = T, raster.dpi = c(5000, 5000)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          plot.title = element_text(size = 20, face = 'plain'),
          legend.text=element_text(size=20)) + NoAxes()
print(umap_plot)
ggsave(umap_plot, file = paste0(dir_plots, name, '_withlegend.pdf'), width = width, height = height)

name <- 'umap_tissue'
grouping <- 'Tissue'
umap_plot <- DimPlot(obj, reduction = 'umap', group.by = grouping, pt.size = 7, label = TRUE, repel  = TRUE, cols = c('lightgrey', '#6F0EAD', '#D3556E'), label.size = 7, raster = T, raster.dpi = c(5000, 5000)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          plot.title = element_text(size = 20, face = 'plain'),
          legend.text=element_text(size=20)) + NoAxes() + NoLegend()
print(umap_plot)
ggsave(umap_plot, file = paste0(dir_plots, name, '.pdf'), width = width, height = height)

umap_plot <- DimPlot(obj, reduction = 'umap', group.by = grouping, pt.size = 7, label = TRUE, repel  = TRUE, cols = c('lightgrey', '#6F0EAD', '#D3556E'), label.size = 7, raster = T, raster.dpi = c(5000, 5000)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          plot.title = element_text(size = 20, face = 'plain'),
          legend.text=element_text(size=20)) + NoAxes()
print(umap_plot)
ggsave(umap_plot, file = paste0(dir_plots, name, '_withlegend.pdf'), width = width, height = height)

name <- 'umap_tumor_normal'
grouping <- 'TumorNormal'
umap_plot <- DimPlot(obj, reduction = 'umap', group.by = grouping, pt.size = 7, label = TRUE, repel  = TRUE, label.size = 7, raster = T, raster.dpi = c(5000, 5000)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          plot.title = element_text(size = 20, face = 'plain'),
          legend.text=element_text(size=20)) + NoAxes() + NoLegend()
print(umap_plot)
ggsave(umap_plot, file = paste0(dir_plots, name, '.pdf'), width = width, height = height)

umap_plot <- DimPlot(obj, reduction = 'umap', group.by = grouping, pt.size = 7, label = TRUE, repel  = TRUE, label.size = 7, raster = T, raster.dpi = c(5000, 5000)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          plot.title = element_text(size = 20, face = 'plain'),
          legend.text=element_text(size=20)) + NoAxes()
print(umap_plot)
ggsave(umap_plot, file = paste0(dir_plots, name, '_withlegend.pdf'), width = width, height = height)

## Cell Cycle

In [ ]:
DefaultAssay(obj) <- 'RNA'

In [ ]:
# A list of cell cycle markers, from Tirosh et al, 2015, is loaded with Seurat.  We can
# segregate this list into markers of G2/M phase and markers of S phase
s.genes <- cc.genes$s.genes
g2m.genes <- cc.genes$g2m.genes

obj <- CellCycleScoring(obj, s.features = s.genes, g2m.features = g2m.genes, set.ident = TRUE)

In [ ]:
name <- 'umap_cell_phase'
grouping <- 'Phase'
umap_plot <- DimPlot(obj, reduction = 'umap', group.by = grouping, pt.size = 7, label = TRUE, repel  = TRUE, label.size = 7, raster = T, raster.dpi = c(5000, 5000)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          plot.title = element_text(size = 20, face = 'plain'),
          legend.text=element_text(size=20)) + NoAxes() + NoLegend()
print(umap_plot)
ggsave(umap_plot, file = paste0(dir_plots, name, '.pdf'), width = width, height = height)

umap_plot <- DimPlot(obj, reduction = 'umap', group.by = grouping, pt.size = 7, label = TRUE, repel  = TRUE, label.size = 7, raster = T, raster.dpi = c(5000, 5000)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          plot.title = element_text(size = 20, face = 'plain'),
          legend.text=element_text(size=20)) + NoAxes()
print(umap_plot)
ggsave(umap_plot, file = paste0(dir_plots, name, '_withlegend.pdf'), width = width, height = height)

In [ ]:
set_figsize(default_width, default_width)
ggplot(obj@meta.data, aes_string(x="CellType", fill="Phase")) +
    geom_bar(position="fill")

## Sample Proportions of Each Cluster

In [ ]:
unique(obj@meta.data$Tissue)

In [ ]:
set_figsize(1.5*default_width, default_width)

df <- obj@meta.data
df$clusters <- df$CellType
clusters <- unique(df$clusters)
dis_state <- 'DLBCL'
df$dis <- df$Tissue
df$dis <- factor(df$dis, levels = c('Tonsil', 'FL', 'DLBCL'))

#determine where are the most cells from HL
order_df <- data.frame(matrix(NA, ncol = 2, nrow = length(clusters)))
colnames(order_df) <- c('cluster', 'dis')
order_df$cluster <- clusters
for(i in 1:nrow(order_df)){
    order_df$dis[i] <- nrow(filter(df, dis == dis_state & clusters == order_df$cluster[i])) / nrow(filter(df, clusters == order_df$cluster[i]))
}

order_df <- order_df[order(order_df$dis), ]
order_list <- order_df$cluster

#order the clusters in the df 
df$clusters <- factor(df$clusters, levels = order_list)

#plot the type of disease state
plot <- ggplot(df, aes_string(x="clusters", fill="dis")) +
    geom_bar(position="fill") + 
    theme(
        plot.title = element_text(hjust = 0.45),
        text = element_text(size=25),
        panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"),
        axis.text.x = element_text(angle = 55, vjust = 1, hjust=1, colour = 'black')) +
        scale_fill_manual('legend', values = c('lightgrey', '#6F0EAD', '#D3556E'))+ 
        ylab('Fraction')+
    ggtitle("Relative")
plot 
ggsave(plot, file = paste0(dir_plots, 'relative_fraction_percluster.pdf'), width = 1.5*default_width, height = default_width)
set_figsize(2*default_width, default_width)




#plot per patient
#determine colors


plot <- ggplot(df, aes_string(x="clusters", fill="Patient")) +
    geom_bar(position="fill") + 
    theme(
        plot.title = element_text(hjust = 0.45),
        text = element_text(size=25),
        panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"),
        axis.text.x = element_text(angle = 55, vjust = 1, hjust=1, colour = 'black')) +
        ylab('Fraction') +
    ggtitle("Relative")
plot 
ggsave(plot, file = paste0(dir_plots, 'relative_fraction_percluster_patient.pdf'), width = 2*default_width, height = default_width)








#plot per disease state
set_figsize(1.5*default_width, default_width)

#determine the order of clusters by fraction within the disease state
order_df <- data.frame(matrix(NA, ncol = 2, nrow = length(clusters)))
colnames(order_df) <- c('cluster', 'dis')
order_df$cluster <- clusters
for(i in 1:nrow(order_df)){
    order_df$dis[i] <- nrow(filter(df, dis == dis_state & clusters == order_df$cluster[i])) / nrow(filter(df, dis == dis_state))
}

order_df <- order_df[order(order_df$dis), ]
order_list <- order_df$cluster

#order the clusters in the df 
df$clusters <- factor(df$clusters, levels = order_list)

#determine colors
c25 <- c(
  "dodgerblue2", "#E31A1C", # red
  "green4",
  "#6A3D9A", # purple
  "#FF7F00", # orange
  "black", "gold1",
  "skyblue2", "#FB9A99", # lt pink
  "palegreen2",
  "#CAB2D6", # lt purple
  "#FDBF6F", # lt orange
  "gray70", "khaki2",
  "maroon", "orchid1", "deeppink1", "blue1", "steelblue4",
  "darkturquoise", "green1", "yellow4", "yellow3",
  "darkorange4", "brown"
)
cols <- c25[1:length(clusters)]



plot <- ggplot(df, aes_string(x="dis", fill="clusters")) +
    geom_bar(position="fill") + 
    theme(
        plot.title = element_text(hjust = 0.45),
        text = element_text(size=25),
        panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"),
        axis.text.x = element_text(angle = 55, vjust = 1, hjust=1, colour = 'black')) +
        ylab('Fraction')+ 
    #scale_fill_manual('legend', values = cols)+
    ggtitle("Relative")
plot 
ggsave(plot, file = paste0(dir_plots, 'relative_fraction_perdis.pdf'), width = 1.5*default_width, height = default_width)

# Plot expressionof markers of interest

In [ ]:
#build an average heatmap obj
obj@meta.data$grouping <- paste0(obj@meta.data$CellType, '_', obj@meta.data$Tissue)

obj@meta.data$grouping <- factor(obj@meta.data$grouping, levels = c('B cells_Tonsil', 'B cells_FL', 'B cells_DLBCL', 'Plasma cells_Tonsil', 'Plasma cells_DLBCL', 'Monocytes and Macrophages_DLBCL',
                                                                   'T cells CD4_Tonsil', 'T cells CD4_FL', 'T cells CD4_DLBCL', 'TFH_Tonsil', 'TFH_FL', 'TFH_DLBCL', 
                                                                   'Tregs_Tonsil', 'Tregs_FL', 'Tregs_DLBCL', 'T cells CD8_Tonsil', 'T cells CD8_FL', 'T cells CD8_DLBCL',
                                                                   'NK cells_FL', 'NK cells_DLBCL', 'Others_FL', 'Others_DLBCL'))
obj$grouping <-  factor(obj$grouping, levels = c('B cells_Tonsil', 'B cells_FL', 'B cells_DLBCL', 'Plasma cells_Tonsil', 'Plasma cells_DLBCL', 'Monocytes and Macrophages_DLBCL',
                                                                   'T cells CD4_Tonsil', 'T cells CD4_FL', 'T cells CD4_DLBCL', 'TFH_Tonsil', 'TFH_FL', 'TFH_DLBCL', 
                                                                   'Tregs_Tonsil', 'Tregs_FL', 'Tregs_DLBCL', 'T cells CD8_Tonsil', 'T cells CD8_FL', 'T cells CD8_DLBCL',
                                                                   'NK cells_FL', 'NK cells_DLBCL', 'Others_FL', 'Others_DLBCL'))

Idents(obj) <- 'grouping'
obj.average <- AverageExpression(obj, assay = "RNA", return.seurat = TRUE)
levels(obj.average)

In [ ]:
#heatmap of markers


width <- 18
height <- 12
name <- 'selcted_markers_hm'
options(repr.plot.width = width, repr.plot.height = height)
markers <- c('CD86',
             'CD19',
            'MS4A1',
            'CD22')
plot <- DoHeatmap(obj.average, features = markers, draw.lines = FALSE, size = 4, raster = FALSE) + coord_equal()+
        theme(text = element_text(size = 20, face = "plain", colour = 'black'),
             axis.text.y=element_text(colour="black", size = 18)) + 
        scale_fill_gradient2(low = '#2881C1', mid = "white", high = "#A20606", na.value = 'white')
plot
ggsave(plot, file = paste0(dir_plots, name, '_coordequal.pdf'), width = width, height = height)

plot <- DoHeatmap(obj.average, features = markers, draw.lines = FALSE, size = 5, raster = FALSE) +
        theme(text = element_text(size = 20, face = "plain", colour = 'black'),
             axis.text.y=element_text(colour="black", size = 18)) +
        scale_fill_gradient2(low = '#2881C1', mid = "white", high = "#A20606", na.value = 'white')
plot
ggsave(plot, file = paste0(dir_plots, name, '_coordunequal.pdf'), width = width, height = height)

In [ ]:
saveRDS(obj, file = './diffuseb/obj_umap_clustered.rds')

# DGE Axis: DLBCL vs Tonsil

In [ ]:
# type in the parameters
dge_dir <- paste0(dir_plots, 'DGE_DLBCL/')
dir.create(dge_dir)
Idents(obj) <- 'CellType'
cluster_of_interest <- levels(obj)
group1 <- 'DLBCL'
group2 <- 'Tonsil'

#create a clustering column for the heatmap
obj@meta.data$cluster_dis <- obj@meta.data$grouping
levels_hm <- levels(obj@meta.data$grouping)

#for subset
object <- obj
Idents(object) <- 'CellType'
levels(object) <- c('B cells', 'Plasma cells', 'Monocytes and Macrophages', 'T cells CD4', 'TFH', 'Tregs', 'T cells CD8', 'NK cells', 'Others')

#for dge
grouping_dge <- 'Tissue'
cutoff_expression <- 0.05 #for the DGE min.pct
logfc_cutoff <- 0.02 #for the DGE analysis
padj_cutoff <- 0.05 #for the genes coming into the GSEA analysis
features <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC|^TRBC", x = rownames(object), value = TRUE)
features <- features[!(features%in%markers.remove)]

#for volcano
#colours_diagnosis_groups <- c('#5cadfb', '#1d9d01', '#ff8a01', '#bebebe')
colour1 <- '#D3556E'
colour2 <- 'lightgrey'
number_of_genes <- 20
mhc2_genes <- rownames(obj)[grep(rownames(obj), pattern = '^HLA-D')]
genes_of_interest_ihb <- c('CD28', 'CTLA4', 'CD80', 'CD86',
                      mhc2_genes, 'LAG3',
                      'CD274', 'PDCD1LG2', 'PDCD1', #PDL1, PDL2, PD1
                      'TNFRSF14', 'BTLA', 'CD160', #HVEM, BTLA, CD150
                      'CEACAM1', 'LGALS9', 'HMGB1', 'HAVCR2', #CEACAM1, Galectin9
                      'NECTIN2', 'NECTIN3', 'PVR', 'TIGIT', 'CD226') #CD112, CD113, CD155
genes_of_interest_act <- c('CD48', 'CD58', 'CD2',
                          'TNFSF15', 'TNFRSF25', #TL1 DR3
                          'TNFSF18', 'TNFRSF18', #GITRL GITR
                           'TNFSF9', 'TNFRSF9', #41BBL 41BB
                          'ICOSLG', 'ICOS', 
                           'TNFSF4', 'TNFRSF4', #OX40L OX40
                           'TNFSF8', 'TNFRSF8', #CD30L CD30
                            'CD40LG', 'CD40', 
                           'CD70', 'CD27'
                          )


#for PEA
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016')

#for heatmap
object_hm <- obj
Idents(object_hm) <- 'cluster_dis'
levels(object_hm) <- levels_hm
object_av <- AverageExpression(object_hm, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

de_general_all <- c()

#here starts the loop with clusters of interest
for(cl in 1:length(cluster_of_interest)){
    tissue_types <- unique(filter(obj@meta.data, CellType == cluster_of_interest[cl])[[grouping_dge]])
    
    if((group1 %in% tissue_types) & (group2 %in% tissue_types)){
    #create the dir with output
    dir_path <- paste0(dge_dir, cluster_of_interest[cl])
    dir.create(dir_path)
    
    object_dge <- subset(object, idents = cluster_of_interest[cl])
    
    de_genes <- FindMarkers(object = object_dge, features = features, only.pos = FALSE, group.by = grouping_dge, ident.1 = group1, ident.2 = group2, min.pct = cutoff_expression, logfc.threshold = logfc_cutoff)
    de_genes$genes <- rownames(de_genes)
    
    
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    de_genes <- rbind(de_positive, de_negative)
    
    de_genes$cell.type <- cluster_of_interest[cl]
    de_general_all <- rbind(de_general_all, de_genes)
    
    
    #build and save the volcano plot
   #or additional genes of interest
    additional_markers_ihb <- genes_of_interest_ihb[genes_of_interest_ihb %in% de_genes$genes]
    additional_markers_act <- genes_of_interest_act[genes_of_interest_act %in% de_genes$genes]

     volcano <- ggplot(de_genes, aes(x = avg_log2FC, y = -log10(p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(padj_cutoff), color ="grey", linetype ="dashed") +
        geom_point(data = de_genes,
                    color = "grey", alpha = 0.5) +
        geom_point(data = subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:number_of_genes, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:number_of_genes, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, p_val_adj < padj_cutoff & genes %in% additional_markers_act), #iclude activatory genes
                    fill = '#2881C1', alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, p_val_adj < padj_cutoff & genes %in% additional_markers_ihb), #include inhibitory genes
                    fill = '#9A0000', alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data=rbind(subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:number_of_genes, ], subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:number_of_genes, ]), max.overlaps = 50, aes(label = genes))+
        geom_text_repel(data=subset(de_genes, p_val_adj < padj_cutoff & genes %in% additional_markers_act), max.overlaps = 50, aes(label = genes), colour = '#003366')+
        geom_text_repel(data=subset(de_genes, p_val_adj < padj_cutoff & genes %in% additional_markers_ihb), max.overlaps = 50, aes(label = genes), colour = '#9A0000')+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none", 
              plot.title = element_text(size = 15, hjust = 0.5)) + ggtitle(paste0(group2, ' (left)', ' vs ', group1, ' (right)')) + 
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)")
    ggsave(volcano, file = paste0(dir_path, '/', group1, '_vs_', group2, '_volcano_plot.pdf'), height = 6, width = 10)
    
    #prepare the subset of genes for PEA (filter for p value)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[de_positive$p_val_adj < padj_cutoff, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[de_negative$p_val_adj < padj_cutoff, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    
    de_all <- rbind(de_positive, de_negative)
    #save the de_all 
    write.csv(de_all, file = paste0(dir_path, '/', group1,  '_vs_', group2, '_de_list.csv'))
    
    #top 30 sorted
    if(length(de_positive$genes) > number_of_genes){
    ordered_genes_plot <- de_positive$genes[1:number_of_genes]
    } else {ordered_genes_plot <- de_positive$genes}
    
    
    if(length(ordered_genes_plot) > 0){
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group1, '_heatmap.pdf'), width = 30, height = 20)
    ggsave(heatmap, file = paste0(dir_path, '/', group1, '_heatmap.eps'), width = 30, height = 20)
    
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E")) + coord_equal()
    ggsave(heatmap, file = paste0(dir_path, '/', group1, '_heatmap_coordequal.pdf'), width = 30, height = 20)
    ggsave(heatmap, file = paste0(dir_path, '/', group1, '_heatmap_coordequal.eps'), width = 30, height = 20)
    }
    

    
    #top 30 sorted
    if(length(de_negative$genes) > number_of_genes){
    ordered_genes_plot <- de_negative$genes[1:number_of_genes]
    } else {ordered_genes_plot <- de_negative$genes}
    
    if(length(ordered_genes_plot) > 0){
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.pdf'), width = 30, height = 20)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.eps'), width = 30, height = 20)
    
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E")) + coord_equal()
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_coordequal.pdf'), width = 30, height = 20)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_coordequal.eps'), width = 30, height = 20)
    }
    
    #start the enrichr PEA 
    for(db in 1:length(databases_list)){
        enriched <- enrichr(de_positive$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group1, '_', databases_list[db], '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group1, databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group1, databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_path, '/', group1, '_', databases_list[db], '_PEA.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group1, '_', databases_list[db], '_',  '_HEATMAP_genes', '.pdf'), width = 30, height = 20)
        }
        
        
        #perform the enrichment on negative genes
        enriched <- enrichr(de_negative$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the negative enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_pea_list.csv'))
            
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_negative <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
            } else {
            plot_negative <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
        }
        ggsave(plot_negative, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_PEA','.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
        }
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_HEATMAP_genes', '.pdf'), width = 30, height = 20)
        }
        }
    #end of the einrichr loop
}
}
#end of the clusters loop

write.csv(de_general_all, file = paste0(dge_dir, 'de_general_all.csv'))

## Create a heatmap of overall markers FC and p values from inhibitory and activatroy panel

In [ ]:
genes_of_interest_ihb <- c('CD86', 'CD80','CTLA4', 
                      'HLA-DRA', 'LAG3',
                      'CD274', 'PDCD1LG2', 'PDCD1', #PDL1, PDL2, PD1
                      'TNFRSF14', 'BTLA', 'CD160', #HVEM, BTLA, CD150
                      'CEACAM1', 'LGALS9', 'HMGB1', 'HAVCR2', #CEACAM1, Galectin9
                      'NECTIN2', 'NECTIN3', 'PVR', 'TIGIT', 'CD226') #CD112, CD113, CD155
genes_of_interest_act <- c(
                            'CD86', 'CD80', 'CD28', 
                            'CD48', 'CD58', 'CD2',
                          'TNFSF15', 'TNFRSF25', #TL1 DR3
                          'TNFSF18', 'TNFRSF18', #GITRL GITR
                           'TNFSF9', 'TNFRSF9', #41BBL 41BB
                          'ICOSLG', 'ICOS', 
                           'TNFSF4', 'TNFRSF4', #OX40L OX40
                           'TNFSF8', 'TNFRSF8', #CD30L CD30
                            'CD40', 'CD40LG', 
                           'CD70', 'CD27'
                          )

In [ ]:
de_general <- read.csv(file = 'pathway/Lymphom/diffuseb/outs/DGE_DLBCL/de_general_all.csv')

In [ ]:
#fill in
clusters <- levels(obj)
genes <- genes_of_interest_ihb
de_output <- de_general


#create a data.frame
df <- data.frame(matrix(NA, ncol = 2, nrow = length(clusters) * length(genes)))
colnames(df) <- c('genes', 'clusters')

df_genes <- c()
for(i in 1:length(genes)){
    repeated_gene <- rep(x = genes[i], times = length(clusters))
    df_genes <- c(df_genes, repeated_gene)
}

df$genes <- df_genes
df$clusters <- rep(x = clusters, times = length(genes))
df$avg_log2FC <- 0
df$p_val_adj <- 1


for(i in 1:nrow(df)){
    fc <- filter(de_output, cell.type == df$clusters[i] & genes == df$genes[i])$avg_log2FC
    pvalue <- filter(de_output, cell.type == df$clusters[i] & genes == df$genes[i])$p_val_adj
    if(length(fc) > 0){
        df$avg_log2FC[i] <- fc
        df$p_val_adj[i] <- pvalue
    }
}

df$p_val_adj[df$p_val_adj > 0.05] <- 1
df$avg_log2FC[df$p_val_adj > 0.05] <- 0


df$clusters <- factor(df$clusters, levels = clusters)
df$genes <- factor(df$genes, levels = rev(genes))

df$avg_log2FC[df$avg_log2FC == 0] <- NA

In [ ]:
width <- 10
height <- 10
name <- 'inhibitory_markers_heatmap_logfc'
set_figsize(width, height)
plot <- ggplot(df, aes(x = clusters, y = genes, fill = avg_log2FC)) + 
  geom_tile() + 
  cowplot::theme_cowplot() + 
  #grids(linetype = "dashed", size = 0.1) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  ylab('') +
  theme(axis.ticks = element_blank()) +
  scale_fill_gradient2(low = '#2881C1', mid = "white", high = "#A20606", na.value = 'white') + coord_equal()
plot
ggsave(plot, file=paste0(dge_dir, group1, '_vs_', group2, '_', name, '.pdf'), width = width, height = height)

In [ ]:
#fill in activatory panel
clusters <- levels(obj)
genes <- genes_of_interest_act
de_output <- de_general


#create a data.frame
df <- data.frame(matrix(NA, ncol = 2, nrow = length(clusters) * length(genes)))
colnames(df) <- c('genes', 'clusters')

df_genes <- c()
for(i in 1:length(genes)){
    repeated_gene <- rep(x = genes[i], times = length(clusters))
    df_genes <- c(df_genes, repeated_gene)
}

df$genes <- df_genes
df$clusters <- rep(x = clusters, times = length(genes))
df$avg_log2FC <- 0
df$p_val_adj <- 1


for(i in 1:nrow(df)){
    fc <- filter(de_output, cell.type == df$clusters[i] & genes == df$genes[i])$avg_log2FC
    pvalue <- filter(de_output, cell.type == df$clusters[i] & genes == df$genes[i])$p_val_adj
    if(length(fc) > 0){
        df$avg_log2FC[i] <- fc
        df$p_val_adj[i] <- pvalue
    }
}

df$p_val_adj[df$p_val_adj > 0.05] <- 1
df$avg_log2FC[df$p_val_adj > 0.05] <- 0

df$clusters <- factor(df$clusters, levels = clusters)
df$genes <- factor(df$genes, levels = rev(genes))

df$avg_log2FC[df$avg_log2FC == 0] <- NA

In [ ]:
width <- 10
height <- 10
name <- 'activation_markers_heatmap_logfc'
set_figsize(width, height)
plot <- ggplot(df, aes(x = clusters, y = genes, fill = avg_log2FC)) + 
  geom_tile() + 
  cowplot::theme_cowplot() + 
  #grids(linetype = "dashed", size = 0.1) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  ylab('') +
  theme(axis.ticks = element_blank()) +
  scale_fill_gradient2(low = '#2881C1', mid = "white", high = "#A20606", na.value = 'white') + coord_equal()
plot
ggsave(plot, file=paste0(dge_dir, group1, '_vs_', group2, '_', name, '.pdf'), width = width, height = height)

# DGE Axis: FL vs Tonsil

In [ ]:
# type in the parameters
dge_dir <- paste0(dir_plots, 'DGE_FL/')
dir.create(dge_dir)
Idents(obj) <- 'CellType'
cluster_of_interest <- levels(obj)
group1 <- 'FL'
group2 <- 'Tonsil'

#create a clustering column for the heatmap
obj@meta.data$cluster_dis <- obj@meta.data$grouping
levels_hm <- levels(obj@meta.data$grouping)

#for subset
object <- obj
Idents(object) <- 'CellType'
levels(object) <- c('B cells', 'Plasma cells', 'Monocytes and Macrophages', 'T cells CD4', 'TFH', 'Tregs', 'T cells CD8', 'NK cells', 'Others')

#for dge
grouping_dge <- 'Tissue'
cutoff_expression <- 0.05 #for the DGE min.pct
logfc_cutoff <- 0.02 #for the DGE analysis
padj_cutoff <- 0.05 #for the genes coming into the GSEA analysis
features <- rownames(object)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^MT-|^IGKV|^IGLV|^IGHV|^IGH|^IGKC|^TRBC", x = rownames(object), value = TRUE)
features <- features[!(features%in%markers.remove)]

#for volcano
#colours_diagnosis_groups <- c('#5cadfb', '#1d9d01', '#ff8a01', '#bebebe')
colour1 <- '#6F0EAD'
colour2 <- 'lightgrey'
number_of_genes <- 20
mhc2_genes <- rownames(obj)[grep(rownames(obj), pattern = '^HLA-D')]
genes_of_interest_ihb <- c('CD28', 'CTLA4', 'CD80', 'CD86',
                      mhc2_genes, 'LAG3',
                      'CD274', 'PDCD1LG2', 'PDCD1', #PDL1, PDL2, PD1
                      'TNFRSF14', 'BTLA', 'CD160', #HVEM, BTLA, CD150
                      'CEACAM1', 'LGALS9', 'HMGB1', 'HAVCR2', #CEACAM1, Galectin9
                      'NECTIN2', 'NECTIN3', 'PVR', 'TIGIT', 'CD226') #CD112, CD113, CD155
genes_of_interest_act <- c('CD48', 'CD58', 'CD2',
                          'TNFSF15', 'TNFRSF25', #TL1 DR3
                          'TNFSF18', 'TNFRSF18', #GITRL GITR
                           'TNFSF9', 'TNFRSF9', #41BBL 41BB
                          'ICOSLG', 'ICOS', 
                           'TNFSF4', 'TNFRSF4', #OX40L OX40
                           'TNFSF8', 'TNFRSF8', #CD30L CD30
                            'CD40LG', 'CD40', 
                           'CD70', 'CD27'
                          )


#for PEA
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016')

#for heatmap
object_hm <- obj
Idents(object_hm) <- 'cluster_dis'
levels(object_hm) <- levels_hm
object_av <- AverageExpression(object_hm, assay = "RNA", return.seurat = TRUE, verbose = FALSE)

de_general_all <- c()

#here starts the loop with clusters of interest
for(cl in 1:length(cluster_of_interest)){
    tissue_types <- unique(filter(obj@meta.data, CellType == cluster_of_interest[cl])[[grouping_dge]])
    
    if((group1 %in% tissue_types) & (group2 %in% tissue_types)){
    #create the dir with output
    dir_path <- paste0(dge_dir, cluster_of_interest[cl])
    dir.create(dir_path)
    
    object_dge <- subset(object, idents = cluster_of_interest[cl])
    
    de_genes <- FindMarkers(object = object_dge, features = features, only.pos = FALSE, group.by = grouping_dge, ident.1 = group1, ident.2 = group2, min.pct = cutoff_expression, logfc.threshold = logfc_cutoff)
    de_genes$genes <- rownames(de_genes)
    
    
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    de_genes <- rbind(de_positive, de_negative)
    
    de_genes$cell.type <- cluster_of_interest[cl]
    de_general_all <- rbind(de_general_all, de_genes)
    
    
    #build and save the volcano plot
   #or additional genes of interest
    additional_markers_ihb <- genes_of_interest_ihb[genes_of_interest_ihb %in% de_genes$genes]
    additional_markers_act <- genes_of_interest_act[genes_of_interest_act %in% de_genes$genes]

     volcano <- ggplot(de_genes, aes(x = avg_log2FC, y = -log10(p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(padj_cutoff), color ="grey", linetype ="dashed") +
        geom_point(data = de_genes,
                    color = "grey", alpha = 0.5) +
        geom_point(data = subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:number_of_genes, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:number_of_genes, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, p_val_adj < padj_cutoff & genes %in% additional_markers_act), #iclude activatory genes
                    fill = '#2881C1', alpha = 1, shape=21, size= 2.5) +
        geom_point(data = subset(de_genes, p_val_adj < padj_cutoff & genes %in% additional_markers_ihb), #include inhibitory genes
                    fill = '#9A0000', alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data=rbind(subset(de_genes, avg_log2FC > 0 & p_val_adj < padj_cutoff)[1:number_of_genes, ], subset(de_genes, avg_log2FC < 0 & p_val_adj < padj_cutoff)[1:number_of_genes, ]), max.overlaps = 50, aes(label = genes))+
        geom_text_repel(data=subset(de_genes, p_val_adj < padj_cutoff & genes %in% additional_markers_act), max.overlaps = 50, aes(label = genes), colour = '#003366')+
        geom_text_repel(data=subset(de_genes, p_val_adj < padj_cutoff & genes %in% additional_markers_ihb), max.overlaps = 50, aes(label = genes), colour = '#9A0000')+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none", 
              plot.title = element_text(size = 15, hjust = 0.5)) + ggtitle(paste0(group2, ' (left)', ' vs ', group1, ' (right)')) + 
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)")
    ggsave(volcano, file = paste0(dir_path, '/', group1, '_vs_', group2, '_volcano_plot.pdf'), height = 6, width = 10)
    
    #prepare the subset of genes for PEA (filter for p value)
    #subset positive genes
    de_positive <- de_genes[de_genes$avg_log2FC > 0, ]
    de_positive <- de_positive[de_positive$p_val_adj < padj_cutoff, ]
    de_positive <- de_positive[order(-de_positive$avg_log2FC), ]
    
    #subset negative genes
    de_negative <- de_genes[de_genes$avg_log2FC < 0, ]
    de_negative <- de_negative[de_negative$p_val_adj < padj_cutoff, ]
    de_negative <- de_negative[order(de_negative$avg_log2FC), ]
    
    de_all <- rbind(de_positive, de_negative)
    #save the de_all 
    write.csv(de_all, file = paste0(dir_path, '/', group1,  '_vs_', group2, '_de_list.csv'))
    
    #top 30 sorted
    if(length(de_positive$genes) > number_of_genes){
    ordered_genes_plot <- de_positive$genes[1:number_of_genes]
    } else {ordered_genes_plot <- de_positive$genes}
    
    
    if(length(ordered_genes_plot) > 0){
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group1, '_heatmap.pdf'), width = 30, height = 20)
    ggsave(heatmap, file = paste0(dir_path, '/', group1, '_heatmap.eps'), width = 30, height = 20)
    
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E")) + coord_equal()
    ggsave(heatmap, file = paste0(dir_path, '/', group1, '_heatmap_coordequal.pdf'), width = 30, height = 20)
    ggsave(heatmap, file = paste0(dir_path, '/', group1, '_heatmap_coordequal.eps'), width = 30, height = 20)
    }
    

    
    #top 30 sorted
    if(length(de_negative$genes) > number_of_genes){
    ordered_genes_plot <- de_negative$genes[1:number_of_genes]
    } else {ordered_genes_plot <- de_negative$genes}
    
    if(length(ordered_genes_plot) > 0){
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.pdf'), width = 30, height = 20)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap.eps'), width = 30, height = 20)
    
    heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, draw.lines = FALSE, size = 7, raster = FALSE) + 
    theme(
        text = element_text(size = 19, colour = 'black', face = 'plain'),
        axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) +
    scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E")) + coord_equal()
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_coordequal.pdf'), width = 30, height = 20)
    ggsave(heatmap, file = paste0(dir_path, '/', group2, '_heatmap_coordequal.eps'), width = 30, height = 20)
    }
    
    #start the enrichr PEA 
    for(db in 1:length(databases_list)){
        enriched <- enrichr(de_positive$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group1, '_', databases_list[db], '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group1, databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group1, databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_path, '/', group1, '_', databases_list[db], '_PEA.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group1, '_', databases_list[db], '_',  '_HEATMAP_genes', '.pdf'), width = 30, height = 20)
        }
        
        
        #perform the enrichment on negative genes
        enriched <- enrichr(de_negative$gene, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the negative enriched pathways
        write.csv(enriched, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_pea_list.csv'))
            
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        if(nrow(enriched) > 0){
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_negative <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
            } else {
            plot_negative <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(group2, databases_list[db]))
        }
        ggsave(plot_negative, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_PEA','.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            genes_to_plot
        }
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, draw.lines = FALSE, raster = FALSE) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_path, '/', group2, '_', databases_list[db], '_HEATMAP_genes', '.pdf'), width = 30, height = 20)
        }
        }
    #end of the einrichr loop
}
}
#end of the clusters loop

write.csv(de_general_all, file = paste0(dge_dir, 'de_general_all.csv'))

## Create a heatmap of overall markers FC and p values from inhibitory and activatroy panel

In [ ]:
genes_of_interest_ihb <- c('CD86', 'CD80','CTLA4', 
                      'HLA-DRA', 'LAG3',
                      'CD274', 'PDCD1LG2', 'PDCD1', #PDL1, PDL2, PD1
                      'TNFRSF14', 'BTLA', 'CD160', #HVEM, BTLA, CD150
                      'CEACAM1', 'LGALS9', 'HMGB1', 'HAVCR2', #CEACAM1, Galectin9
                      'NECTIN2', 'NECTIN3', 'PVR', 'TIGIT', 'CD226') #CD112, CD113, CD155
genes_of_interest_act <- c(
                            'CD86', 'CD80', 'CD28', 
                            'CD48', 'CD58', 'CD2',
                          'TNFSF15', 'TNFRSF25', #TL1 DR3
                          'TNFSF18', 'TNFRSF18', #GITRL GITR
                           'TNFSF9', 'TNFRSF9', #41BBL 41BB
                          'ICOSLG', 'ICOS', 
                           'TNFSF4', 'TNFRSF4', #OX40L OX40
                           'TNFSF8', 'TNFRSF8', #CD30L CD30
                            'CD40', 'CD40LG', 
                           'CD70', 'CD27'
                          )

In [ ]:
de_general <- read.csv(file = 'pathway/Lymphom/diffuseb/outs/DGE_FL/de_general_all.csv')

In [ ]:
#fill in
clusters <- levels(obj)
genes <- genes_of_interest_ihb
de_output <- de_general


#create a data.frame
df <- data.frame(matrix(NA, ncol = 2, nrow = length(clusters) * length(genes)))
colnames(df) <- c('genes', 'clusters')

df_genes <- c()
for(i in 1:length(genes)){
    repeated_gene <- rep(x = genes[i], times = length(clusters))
    df_genes <- c(df_genes, repeated_gene)
}

df$genes <- df_genes
df$clusters <- rep(x = clusters, times = length(genes))
df$avg_log2FC <- 0
df$p_val_adj <- 1


for(i in 1:nrow(df)){
    fc <- filter(de_output, cell.type == df$clusters[i] & genes == df$genes[i])$avg_log2FC
    pvalue <- filter(de_output, cell.type == df$clusters[i] & genes == df$genes[i])$p_val_adj
    if(length(fc) > 0){
        df$avg_log2FC[i] <- fc
        df$p_val_adj[i] <- pvalue
    }
}

df$p_val_adj[df$p_val_adj > 0.05] <- 1
df$avg_log2FC[df$p_val_adj > 0.05] <- 0


df$clusters <- factor(df$clusters, levels = clusters)
df$genes <- factor(df$genes, levels = rev(genes))

df$avg_log2FC[df$avg_log2FC == 0] <- NA

In [ ]:
width <- 10
height <- 10
name <- 'inhibitory_markers_heatmap_logfc'
set_figsize(width, height)
plot <- ggplot(df, aes(x = clusters, y = genes, fill = avg_log2FC)) + 
  geom_tile() + 
  cowplot::theme_cowplot() + 
  #grids(linetype = "dashed", size = 0.1) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  ylab('') +
  theme(axis.ticks = element_blank()) +
  scale_fill_gradient2(low = '#2881C1', mid = "white", high = "#A20606", na.value = 'white') + coord_equal()
plot
ggsave(plot, file=paste0(dge_dir, group1, '_vs_', group2, '_', name, '.pdf'), width = width, height = height)

In [ ]:
#fill in activatory panel
clusters <- levels(obj)
genes <- genes_of_interest_act
de_output <- de_general


#create a data.frame
df <- data.frame(matrix(NA, ncol = 2, nrow = length(clusters) * length(genes)))
colnames(df) <- c('genes', 'clusters')

df_genes <- c()
for(i in 1:length(genes)){
    repeated_gene <- rep(x = genes[i], times = length(clusters))
    df_genes <- c(df_genes, repeated_gene)
}

df$genes <- df_genes
df$clusters <- rep(x = clusters, times = length(genes))
df$avg_log2FC <- 0
df$p_val_adj <- 1


for(i in 1:nrow(df)){
    fc <- filter(de_output, cell.type == df$clusters[i] & genes == df$genes[i])$avg_log2FC
    pvalue <- filter(de_output, cell.type == df$clusters[i] & genes == df$genes[i])$p_val_adj
    if(length(fc) > 0){
        df$avg_log2FC[i] <- fc
        df$p_val_adj[i] <- pvalue
    }
}

df$p_val_adj[df$p_val_adj > 0.05] <- 1
df$avg_log2FC[df$p_val_adj > 0.05] <- 0

df$clusters <- factor(df$clusters, levels = clusters)
df$genes <- factor(df$genes, levels = rev(genes))

df$avg_log2FC[df$avg_log2FC == 0] <- NA

In [ ]:
width <- 10
height <- 10
name <- 'activation_markers_heatmap_logfc'
set_figsize(width, height)
plot <- ggplot(df, aes(x = clusters, y = genes, fill = avg_log2FC)) + 
  geom_tile() + 
  cowplot::theme_cowplot() + 
  #grids(linetype = "dashed", size = 0.1) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  ylab('') +
  theme(axis.ticks = element_blank()) +
  scale_fill_gradient2(low = '#2881C1', mid = "white", high = "#A20606", na.value = 'white') + coord_equal()
plot
ggsave(plot, file=paste0(dge_dir, group1, '_vs_', group2, '_', name, '.pdf'), width = width, height = height)